In [1]:
#tensorflow
import tensorflow as tf
from tensorflow import keras

#sklearn
import sklearn
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Common imports
import numpy as np
import pandas as pd
import os
import os.path
import urllib

# To plot pretty figures
import matplotlib.pyplot as plt

In [2]:
from polygon import RESTClient

In [3]:
from datetime import date, datetime
from typing import Any, Optional
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

markets = ['crypto', 'stocks', 'fx']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str='xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2', timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

In [4]:
client = MyRESTClient(['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'])

In [5]:
class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

In [6]:
client = MyRESTClient(['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'])
df = client.get_tickers(market='crypto')
df

,ticker,name,market,active
0,X:1INCHUSD,1inch,crypto,True
1,X:AAVEUSD,Aave,crypto,True
2,X:ACATUSD,Alphacat,crypto,True
3,X:ACHUSD,Alchemy Pay,crypto,True
4,X:ACTUSD,Achain,crypto,True
...,...,...,...,...
92,X:ZENUSD,Horizen,crypto,True
93,X:ZILUSD,Zilliqa,crypto,True
94,X:ZMTUSD,Zipmex,crypto,True
95,X:ZRXUSD,0x,crypto,True


In [467]:
class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

    def get_bars(self, market:str=None, ticker:str=None, multiplier:int=1,
                 timespan:str='hour', from_:date=None, to:date=None) -> pd.DataFrame:

        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        if ticker is None:
            raise Exception('Ticker must not be None.')

        from_ = from_ if from_ else date(2000,1,1)
        to = to if to else date.today()

        if market == 'crypto':
            resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          from_.strftime('%Y-%m-%d'), to.strftime('%Y-%m-%d'),
                                          limit=50000)
            df = pd.DataFrame(resp.results)
            last_minute = 0
            while resp.results[-1]['t'] > last_minute:
                last_minute = resp.results[-1]['t'] # Last minute in response
                last_minute_date = datetime.fromtimestamp(last_minute/1000).strftime('%Y-%m-%d')
                resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          last_minute_date, to.strftime('%Y-%m-%d'),
                                          limit=50000)
                new_bars = pd.DataFrame(resp.results)
                df = df.append(new_bars[new_bars['t'] > last_minute])

            df['date'] = pd.to_datetime(df['t'], unit='ms')
            df = df.rename(columns={'o':'open',
                                    'h':'high',
                                    'l':'low',
                                    'c':'close',
                                    'v':'volume',
                                    'vw':'vwap',
                                    'n':'transactions'})
            df = df[['date','open','high','low','close','volume']]

            return df
        return None

In [642]:
start = datetime(2017,1,1)
end = datetime(2021,12,31)
client = MyRESTClient(['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'])
df1 = client.get_bars(market='crypto', ticker='X:BTCUSD', from_=start,to=end)


In [643]:
data=df1

data

,date,open,high,low,close,volume
0,2017-01-01 00:00:00,973.370,973.40,962.04,967.00000,401.218604
1,2017-01-01 01:00:00,970.970,972.00,962.54,966.78900,362.969012
2,2017-01-01 02:00:00,966.789,971.99,961.99,963.97000,285.893016
3,2017-01-01 03:00:00,967.820,969.47,960.53,968.14000,270.455020
4,2017-01-01 04:00:00,968.180,968.62,960.60,968.10000,221.671967
...,...,...,...,...,...,...
547,2021-12-31 19:00:00,46667.000,46674.91,45530.30,45721.07152,3148.211376
548,2021-12-31 20:00:00,45703.050,46208.92,45650.00,45867.86000,2349.291271
549,2021-12-31 21:00:00,45879.970,46516.01,45773.87,46325.88000,1651.890409
550,2021-12-31 22:00:00,46329.340,46577.00,46246.51,46310.01000,937.168808


In [644]:
## Calculate the MACD and Signal Line indicators
## Calculate the Short Term Exponential Moving Average
ShortEMA = data.close.ewm(span=12, adjust=False).mean() 
## Calculate the Long Term Exponential Moving Average
LongEMA = data.close.ewm(span=26, adjust=False).mean() 
## Calculate the Moving Average Convergence/Divergence (MACD)
data['MACD'] = ShortEMA - LongEMA
## Calcualte the signal line
data['signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

In [645]:
from statistics import median

#classical momentum gauge at t=0 day tenor

ChangeInHigh=data.high.shift(+25)/data.close.shift(+1)
ChangeInLow=data.close.shift(+1)/data.low.shift(+25)
TrailingStd=data.close.shift(+1).rolling(24).std()/data.close.shift(+1).rolling(24).mean()
HighDifferential=ChangeInHigh/TrailingStd
LowDifferential=ChangeInLow/TrailingStd

Momentum=(LowDifferential-HighDifferential)
data['Momentum%']=Momentum/100

In [646]:
from statistics import median

#classical momentum gauge at 1 day tenor (T=1)

ChangeInHighMinus_1=data.high.shift(+26)/data.close.shift(+2)
ChangeInLowMinus_1=data.close.shift(+2)/data.low.shift(+26)
TrailingStdMinus_1=data.close.shift(+2).rolling(24).std()/data.close.shift(+2).rolling(24).mean()
HighDifferentialMinus_1=ChangeInHighMinus_1/TrailingStdMinus_1
LowDifferentialMinus_1=ChangeInLowMinus_1/TrailingStdMinus_1

MomentumMinus_1=(LowDifferentialMinus_1-HighDifferentialMinus_1)
data['Momentum%Minus_1']=Momentum/100

In [647]:
from statistics import median

#classical momentum gauge at 2 day tenor (T=2)

ChangeInHighMinus_2=data.high.shift(+27)/data.close.shift(+3)
ChangeInLowMinus_2=data.close.shift(+3)/data.low.shift(+27)
TrailingStdMinus_2=data.close.shift(+3).rolling(24).std()/data.close.shift(+3).rolling(24).mean()
HighDifferentialMinus_2=ChangeInHighMinus_2/TrailingStdMinus_2
LowDifferentialMinus_2=ChangeInLowMinus_2/TrailingStdMinus_2

MomentumMinus_2=(LowDifferentialMinus_2-HighDifferentialMinus_2)
data['Momentum%Minus_2']=MomentumMinus_2/100

In [648]:
from statistics import median

#classical momentum gauge at 3 day tenor (T=3)

ChangeInHighMinus_3=data.high.shift(+28)/data.close.shift(+4)
ChangeInLowMinus_3=data.close.shift(+4)/data.low.shift(+28)
TrailingStdMinus_3=data.close.shift(+4).rolling(24).std()/data.close.shift(+4).rolling(24).mean()
HighDifferentialMinus_3=ChangeInHighMinus_3/TrailingStdMinus_3
LowDifferentialMinus_3=ChangeInLowMinus_3/TrailingStdMinus_3

MomentumMinus_3=(LowDifferentialMinus_3-HighDifferentialMinus_3)
data['Momentum%Minus_3']=MomentumMinus_3/100

In [649]:
from statistics import median

#classical momentum gauge at 4 day tenor (T=4)

ChangeInHighMinus_4=data.high.shift(+29)/data.close.shift(+5)
ChangeInLowMinus_4=data.close.shift(+5)/data.low.shift(+29)
TrailingStdMinus_4=data.close.shift(+5).rolling(24).std()/data.close.shift(+5).rolling(24).mean()
HighDifferentialMinus_4=ChangeInHighMinus_4/TrailingStdMinus_4
LowDifferentialMinus_4=ChangeInLowMinus_4/TrailingStdMinus_4

MomentumMinus_4=(LowDifferentialMinus_4-HighDifferentialMinus_4)
data['Momentum%Minus_4']=MomentumMinus_4/100

In [340]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES

In [349]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()


df_train = pd.DataFrame()

i=0
for i in tqdm(range(43800)):
    if i <100:
        pass
    else:
        if i==100:
            df_train =data['close'].iloc[0:i]
            model = HWES(df_train, seasonal_periods=24, trend='mul', seasonal='mul').fit()
            forecast_1hr=model.forecast(1)
         
        else:
            df_train =data['close'].iloc[i-100:i]
            model = HWES(df_train, seasonal_periods=24, trend='mul', seasonal='mul').fit()
            forecast_1hr=np.append(forecast_1hr,model.forecast(1))


forecast_1hr

100%|██████████████████████████████████████████████████████████████████████████| 43800/43800 [2:23:44<00:00,  5.08it/s]


array([ 1151.23309649,  1146.61685278,  1149.84794524, ...,
       45922.05858857, 46403.33892316, 46464.23215347])

In [650]:
naive=pd.DataFrame(forecast_1hr)
naive

,0
0,1151.233096
1,1146.616853
2,1149.847945
3,1152.354823
4,1146.127100
...,...
43695,45390.642937
43696,45761.736694
43697,45922.058589
43698,46403.338923


In [651]:
data=data[100:]
data=data.reset_index()
data=data.drop('index',axis=1)
data

,date,open,high,low,close,volume,MACD,signal,Momentum%,Momentum%Minus_1,Momentum%Minus_2,Momentum%Minus_3,Momentum%Minus_4
0,2017-01-05 04:00:00,1145.00,1165.00,1116.39,1155.10000,2935.686882,23.131546,20.780096,0.065283,0.065283,0.063329,0.036941,0.064065
1,2017-01-05 05:00:00,1146.77,1175.00,1110.00,1158.00000,2277.977260,24.167679,21.457613,0.066577,0.066577,0.061863,0.063329,0.036941
2,2017-01-05 06:00:00,1158.00,1174.90,1116.39,1149.98200,2174.642226,24.064436,21.978977,0.061965,0.061965,0.065283,0.061863,0.063329
3,2017-01-05 07:00:00,1158.34,1172.13,1105.59,1144.09900,2007.550802,23.240009,22.231184,0.060069,0.060069,0.066577,0.065283,0.061863
4,2017-01-05 08:00:00,1167.37,1168.99,1105.60,1132.50000,4227.822898,21.403972,22.065741,0.055607,0.055607,0.061965,0.066577,0.065283
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43695,2021-12-31 19:00:00,46667.00,46674.91,45530.30,45721.07152,3148.211376,-144.274296,59.138818,-0.044358,-0.044358,0.004293,0.027541,0.031758
43696,2021-12-31 20:00:00,45703.05,46208.92,45650.00,45867.86000,2349.291271,-242.163020,-1.121550,-0.068747,-0.068747,-0.028161,0.004293,0.027541
43697,2021-12-31 21:00:00,45879.97,46516.01,45773.87,46325.88000,1651.890409,-279.559565,-56.809153,-0.046239,-0.046239,-0.044358,-0.028161,0.004293
43698,2021-12-31 22:00:00,46329.34,46577.00,46246.51,46310.01000,937.168808,-306.938972,-106.835117,-0.023242,-0.023242,-0.068747,-0.044358,-0.028161


In [652]:
data=data.merge(naive,left_index=True,right_index=True)

In [653]:
data.columns

Index([            'date',             'open',             'high',
                    'low',            'close',           'volume',
                   'MACD',           'signal',        'Momentum%',
       'Momentum%Minus_1', 'Momentum%Minus_2', 'Momentum%Minus_3',
       'Momentum%Minus_4',                  0],
      dtype='object')

In [654]:
data.shape

(43700, 14)

In [655]:
data.to_csv('BTC_17_21.csv',index=False)

In [217]:
import random
from collections import deque

In [294]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+list(map('{:.2f}%'.format,n))
def getStockDataVec(key):
    vec = []
    lines = open("BTC_17_21.csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append([float(vec) for vec in line.split(',')[1:]])
        
        #print(vec)
    return vec
def sigmoid(x):
    return 1/(1+np.exp(-x))
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        array1 = np.array(block[i])
        array2 = np.array(block[i + 1])
        res.append(sigmoid(np.subtract(array2, array1)))
    return np.array([res])





In [316]:
from keras.layers import BatchNormalization

class Agent:    
    def __init__(self, state_size, is_eval=False, model_name=""):
            self.state_size = state_size # normalized previous days
            self.action_size = 4 # hold, buy, sell,  sell all
            self.memory = deque(maxlen=720)
            self.inventory = []
            self.skip = skip
            self.trend = trend
            self.model_name = model_name
            self.is_eval = is_eval
            self.gamma = 0.95
            self.epsilon = .25
            self.epsilon_min = 0.01
            self.epsilon_decay = 0.995
            self.model = tf.keras.models.load_model(model_name) if is_eval else self._model()
            
    def _model(self):
        model = keras.models.Sequential()
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=1200, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=600, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=300, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=150, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=75, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(units=25, activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random()<= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(np.array(state))
        return np.argmax(options[0])
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f,epochs=5, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [296]:
data = getStockDataVec(stock_name)
data

[[1145.0,
  1165.0,
  1116.39,
  1155.1,
  2935.6868824899993,
  23.131546041302954,
  20.780096433195304,
  0.0652830840030128,
  0.0652830840030128,
  0.06332863521015639,
  0.036940857787714804,
  0.064065109411782,
  1151.2330964898006],
 [1146.77,
  1175.0,
  1110.0,
  1158.0,
  2277.97725988,
  24.167678523224367,
  21.457612851201116,
  0.06657679573345429,
  0.06657679573345429,
  0.06186297809280738,
  0.06332863521015639,
  0.036940857787714804,
  1146.6168527841835],
 [1158.0,
  1174.9,
  1116.39,
  1149.982,
  2174.64222607,
  24.06443555580404,
  21.978977392121703,
  0.06196501735924038,
  0.06196501735924038,
  0.0652830840030128,
  0.06186297809280738,
  0.06332863521015639,
  1149.847945241877],
 [1158.34,
  1172.13,
  1105.59,
  1144.099,
  2007.5508020400005,
  23.240009463884235,
  22.23118380647421,
  0.06006884525415419,
  0.06006884525415419,
  0.06657679573345429,
  0.0652830840030128,
  0.06186297809280738,
  1152.3548233287447],
 [1167.37,
  1168.99,
  1105.6,

In [297]:
btc = [item[3] for item in data]
btc[1]

1158.0

In [298]:
shape=np.array(data[28916:28928 + 1]).shape

In [299]:
getState(data, 13, window_size + 1)

<ipython-input-294-7f55dc83a67a>:14: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


array([[[5.00000000e-01, 5.00000000e-01, 5.00000000e-01, ...,
         5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
        [5.00000000e-01, 5.00000000e-01, 5.00000000e-01, ...,
         5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
        [5.00000000e-01, 5.00000000e-01, 5.00000000e-01, ...,
         5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
        ...,
        [2.39972146e-03, 1.00000000e+00, 4.47692090e-01, ...,
         5.00493398e-01, 5.00357066e-01, 5.60748398e-01],
        [1.00000000e+00, 7.01531032e-07, 9.99995403e-01, ...,
         4.81682934e-01, 5.00493398e-01, 1.00000000e+00],
        [1.35519666e-17, 1.39960675e-14, 1.86919655e-06, ...,
         5.00813314e-01, 4.81682934e-01, 8.15221333e-14]]])

In [336]:
import math

stock_name = input("Enter stock_name, window_size, Episode_count")
window_size = input()
episode_count = input()
stock_name = str(stock_name)
window_size = int(window_size)
episode_count = int(episode_count)
trend=btc
agent = Agent(window_size)
l = len(data) - 1

batch_size = 10000
np.random.seed(42)
tf.random.set_seed(42)
keras.backend.clear_session()
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    
    agent.inventory = []
    
   
    
   
    for t in range(l):
        action = agent.act(np.array(state))
        price=btc[t]
        
        if t==0:
            Liquidity=100000
            shares=[Liquidity/btc[t]]
        elif t>=1:
            Liquidity=(100000-sum(agent.inventory)+total_profit)
        if btc[t]>=Liquidity:
            action = 0 or 3
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0
        
        
        if action == 1: # buy
            agent.inventory.append(btc[t])
            inventory=len(agent.inventory)
            Liquidity=(100000+total_profit-sum(agent.inventory))
        
            
            print("Buy "+str(price)+"------"+str(Liquidity)+"------"+str(sum(agent.inventory)))
        
        elif action == 2: # buy all
            Liquidity=(100000+total_profit-sum(agent.inventory))
            shares.append(math.trunc(Liquidity/price))
            buyallPrice=price
            LiquidityAll=Liquidity
            agent.inventory.append(btc[t]*shares[-1])
            
            Liquidity=(100000+total_profit-sum(agent.inventory))
            
            print("Buy All "+str(price)+"------"+str(Liquidity)+"------"+str(sum(agent.inventory)))
    
            
        elif action == 3: 
            while len(agent.inventory) >=1 and len(shares)>=1: # sell all
                shares2=LiquidityAll/buyallPrice
                bought_price =  sum(agent.inventory)/(shares2+inventory)
                total_profit += (btc[t] - bought_price)*len(agent.inventory)
                Liquidity=(100000+total_profit-sum(agent.inventory))
               
                agent.inventory.pop(0)
                
                print("Sell All "+"------"+str(price)+"-------"+str(total_profit)+"---------"+str((((total_profit+100000)/100000)/(price/1155.1)))+"--------"+str(len(agent.inventory)))
    
            

            reward = max(total_profit*(((((total_profit+100000)/100000))/(((price)/1155.1)))-1), 0)
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Price: " + str(price)+" Total Profit: "+str(total_profit)+" | Inventory: " + str(sum(agent.inventory))+" " + str(len(agent.inventory))+" | Liquidity: " + str(Liquidity)+" | Alpha: "+str((((total_profit+100000)/100000)/(price/1155.1))))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    
    if e % 10 == 0:
        agent.model.save(str(e))

Enter stock_name, window_size, Episode_countBTC
240
5
Episode 0/5
Buy All 1155.1------661.4000000000087------99338.59999999999
Sell All ------1158.0-------23.642791515208046---------0.9977315180352264--------0
Buy All 1149.982------1125.1907915152115------98898.45199999999
Sell All ------1144.099-------43.62036300826048---------1.010055824551117--------0
Buy All 1073.04------250.90036300825886------99792.72
Sell All ------1085.69-------70.31988721151538---------1.0646798487755988--------0
Buy All 1096.82------259.69988721152185------99810.62


<ipython-input-294-7f55dc83a67a>:14: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Sell All ------969.609--------42.184181819099535---------1.1908024064502365--------0
Buy All 995.49------408.8158181808976------99549.0
Sell All ------954.5--------69.32646888279737---------1.2093234258333523--------0
Buy 1021.0------98909.67353111721------1021.0
Sell All ------980.59-------901.1955568021212---------1.188580048620342--------0
Buy All 977.59------209.42555680211808------100691.77
Sell All ------983.32-------918.31570571126---------1.1854812926785487--------0
Buy 834.952------100083.36370571125------834.952
Buy All 890.0------403.3637057112501------100514.952
Sell All ------885.954-------918.3047758689719---------1.3157650831375698--------1
Sell All ------885.954-------925.6587496426287---------1.3158609636810963--------0
Buy 883.03------100042.62874964262------883.03
Buy 887.78------99154.84874964262------1770.81
Sell All ------890.69-------2676.094927591882---------1.3315649356213877--------1
Sell All ------890.69-------3559.02822124275---------1.3430153420197544------

Sell All ------912.56-------27417.83423552986---------1.6128291874009437--------1
Sell All ------912.56-------28323.72936839119---------1.6242958248600492--------0
Buy All 913.85------384.72936839118483------127939.0
Sell All ------917.99-------28343.403800768276---------1.6149355192351489--------0
Buy 914.96------127428.44380076828------914.96
Buy All 919.3------565.0438007682824------127778.36
Sell All ------920.45-------28353.860306095845---------1.610750655001046--------1
Sell All ------920.45-------28365.64202595034---------1.6108985072972482--------0
Buy 920.46------127445.18202595033------920.46
Buy All 916.0------121.18202595032926------128244.46
Sell All ------916.8-------28368.9079092662---------1.6173530271159837--------1
Sell All ------916.8-------28377.109358214984---------1.6174563592896392--------0
Buy All 922.48------152.3893582149758------128224.72
Sell All ------921.69-------28383.987947809797---------1.6089611960476418--------0
Buy All 920.08------492.8679478097911--

Buy 1010.5------133201.74852072232------1010.5
Sell All ------1013.18-------35217.88233218063---------1.5415836858396517--------0
Buy 1012.02------134205.86233218064------1012.02
Sell All ------1014.516-------36224.84079261805---------1.5510185507133756--------0
Buy All 1029.5------330.84079261805164------135894.0
Sell All ------1035.38-------36240.92426478449---------1.519943321468954--------0
Buy 1039.5------135201.4242647845------1039.5
Sell All ------1042.3-------37275.42731465657---------1.5213167618839085--------0
Buy All 1042.4------721.0273146565305------136554.40000000002
Sell All ------1045.969-------37292.28597274682---------1.516166535787579--------0
Buy All 1042.14------771.9459727467911------136520.34000000003
Sell All ------1054.52-------37318.33235797482---------1.504157395845472--------0
Buy All 1056.58------1019.5123579748324------136298.81999999998
Sell All ------1059.69-------37337.29463658667---------1.4970256304647704--------0
Buy 1057.7------136279.59463658667---

...

Buy All 1789.53------1089.7979618850513------280956.21
Sell All ------1869.17-------182143.80158635587---------1.7435776585992695--------0
Buy All 1790.01------1112.2315863558906------281031.57
Sell All ------1873.8-------182245.88825005002---------1.739898737952998--------0
Buy All 1876.012------844.0882500500302------281401.8
Sell All ------1875.1-------182262.9366420712---------1.7387974940816837--------0
Buy 1934.98------280327.95664207125------1934.98
Buy 1932.011------278395.94564207125------3866.991
Buy 1965.7------276430.24564207124------5832.691
Buy All 1966.2------1162.2456420712406------281100.691
Sell All ------1939.57-------182190.62919519164---------1.6805704139750866--------3
Sell All ------1939.57-------182176.82548530234---------1.6804882067575428--------2
Sell All ------1939.57-------182194.53290868877---------1.6805936623211657--------1
Sell All ------1939.57-------182217.07618628946---------1.68072791754246--------0
Buy 1931.985------280285.09118628944------1931.985

Buy 2845.0------300813.2717818509------5729.37
Buy 2856.61------297956.6617818509------8585.98
Buy All 2793.3------1866.8617818508646------304675.78
Sell All ------2825.0-------206730.01648732202---------1.2541728921929403--------3
Sell All ------2825.0-------206949.45005428244---------1.2550701230361119--------2
Sell All ------2825.0-------207147.62280783278---------1.2558804216117792--------1
Sell All ------2825.0-------207272.75690371334---------1.2563920761043512--------0
Buy 2905.99------304366.76690371335------2905.99
Buy 2995.96------301370.8069037133------5901.95
Buy 2999.98------298370.82690371334------8901.93
Buy All 2897.0------2876.826903713343------304395.93
Sell All ------2880.125-------207305.86339600102---------1.2324777667938744--------3
Sell All ------2880.125-------207412.9436684814---------1.232907221844409--------2
Sell All ------2880.125-------207540.86177767193---------1.2334202489106856--------1
Sell All ------2880.125-------207633.1243897222---------1.233790276

Sell All ------2342.52-------247279.27295245038---------1.7124391176484104--------1
Sell All ------2342.52-------249605.79526214342---------1.7239112328061312--------0
Buy All 2497.772------2415.487262143404------347190.308
Sell All ------2478.47-------249638.6957338437---------1.629503917506215--------0
Buy All 2483.72------1917.8957338437322------347720.8
Sell All ------2490.08-------249693.28176364285---------1.622159568227462--------0
Buy All 2558.0------1805.2817636428517------347888.0
Sell All ------2563.1-------249748.28079295627---------1.5761938244467395--------0
Buy 2615.7------347132.5807929562------2615.7
Sell All ------2613.9-------252343.18594198936---------1.5570282492887708--------0
Buy All 2563.57499------1133.4123119893484------351209.77363
Sell All ------2594.0-------252400.3151606126---------1.5692274635390269--------0
Buy All 2548.4------721.1151606125641------351679.2
Sell All ------2554.0-------252429.389065657---------1.5939357373130008--------0
Buy All 2602.0--

Buy All 2751.01------1072.836250268505------382390.34
Sell All ------2755.87-------283488.2902729612---------1.607359287971847--------1
Sell All ------2755.87-------283520.5829928489---------1.6074946402226509--------0
Buy 2572.799------380947.7839928489------2572.799
Buy All 2574.1------2555.0839928488713------380965.499
Buy All 2555.05------0.033992848882917315------383520.549
Sell All ------2494.99--------284271.4437123705----------0.8531174258500402--------2
Sell All ------2494.99--------660226.6796302288----------2.5936690633664954--------1
Sell All ------2494.99--------659009.2061320731----------2.588032553249342--------0


KeyboardInterrupt: 

In [ ]:
agent.model.save('BTC_AgentModel.h5')

In [ ]:
start = datetime(2022,1,1)

client = MyRESTClient(['xz0v5qmSd2ZFCW1Q8A4rOIsIWtk5cht2'])
df2 = client.get_bars(market='crypto', ticker='X:BTCUSD', from_=start)
data2=df2

In [270]:
## Calculate the MACD and Signal Line indicators
## Calculate the Short Term Exponential Moving Average
ShortEMA = data2.close.ewm(span=1440, adjust=False).mean() 
## Calculate the Long Term Exponential Moving Average
LongEMA = data2.close.ewm(span=43200, adjust=False).mean() 
## Calculate the Moving Average Convergence/Divergence (MACD)
data2['MACD'] = ShortEMA - LongEMA
## Calcualte the signal line
data2['signal'] = data2['MACD'].ewm(span=9, adjust=False).mean()

In [272]:
from statistics import median

#classical momentum gauge at t=0 day tenor

ChangeInHigh=data.high.shift(+25)/data.close.shift(+1)
ChangeInLow=data.close.shift(+1)/data.low.shift(+25)
TrailingStd=data.close.shift(+1).rolling(24).std()/data.close.shift(+1).rolling(24).mean()
HighDifferential=data['ChangeInHigh']/data['TrailingStd']
LowDifferential=data['ChangeInLow']/data['TrailingStd']

Momentum=(LowDifferential-HighDifferential)
data['Momentum%']=Momentum/100

In [273]:
from statistics import median

#classical momentum gauge at 1 day tenor (T=1)

ChangeInHighMinus_1=data.high.shift(+26)/data.close.shift(+2)
ChangeInLowMinus_1=data.close.shift(+2)/data.low.shift(+26)
TrailingStdMinus_1=data.close.shift(+2).rolling(24).std()/data.close.shift(+2).rolling(24).mean()
HighDifferentialMinus_1=ChangeInHighMinus_1/TrailingStdMinus_1
LowDifferentialMinus_1=ChangeInLowMinus_1/TrailingStdMinus_1

MomentumMinus_1=(LowDifferentialMinus_1-HighDifferentialMinus_1)
data['Momentum%Minus_1']=Momentum/100

In [274]:
from statistics import median

#classical momentum gauge at 2 day tenor (T=2)

ChangeInHighMinus_2=data.high.shift(+27)/data.close.shift(+3)
ChangeInLowMinus_2=data.close.shift(+3)/data.low.shift(+27)
TrailingStdMinus_2=data.close.shift(+3).rolling(24).std()/data.close.shift(+3).rolling(24).mean()
HighDifferentialMinus_2=ChangeInHighMinus_2/TrailingStdMinus_2
LowDifferentialMinus_2=ChangeInLowMinus_2/TrailingStdMinus_2

MomentumMinus_2=(LowDifferentialMinus_2-HighDifferentialMinus_2)
data['Momentum%Minus_2']=MomentumMinus_2/100

In [275]:
from statistics import median

#classical momentum gauge at 3 day tenor (T=3)

dataChangeInHighMinus_3=data.high.shift(+28)/data.close.shift(+4)
dataChangeInLowMinus_3=data.close.shift(+4)/data.low.shift(+28)
dataTrailingStdMinus_3=data.close.shift(+4).rolling(24).std()/data.close.shift(+4).rolling(24).mean()
dataHighDifferentialMinus_3=dataChangeInHighMinus_3/dataTrailingStdMinus_3
dataLowDifferentialMinus_3=dataChangeInLowMinus_3/dataTrailingStdMinus_3

MomentumMinus_3=(LowDifferentialMinus_3-HighDifferentialMinus_3)
data['Momentum%Minus_3']=MomentumMinus_3/100

In [276]:
from statistics import median

#classical momentum gauge at 4 day tenor (T=4)

ChangeInHighMinus_4=data.high.shift(+29)/data.close.shift(+5)
ChangeInLowMinus_4=data.close.shift(+5)/data.low.shift(+29)
TrailingStdMinus_4=data.close.shift(+5).rolling(24).std()/data.close.shift(+5).rolling(24).mean()
HighDifferentialMinus_4=ChangeInHighMinus_4/TrailingStdMinus_4
LowDifferentialMinus_4=ChangeInLowMinus_4/TrailingStdMinus_4

MomentumMinus_4=(LowDifferentialMinus_4-HighDifferentialMinus_4)
data['Momentum%Minus_4']=MomentumMinus_4/100

In [ ]:
data2.close.to_csv('BTC22.csv',index=False)

In [ ]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+list(map('{:.2f}%'.format,n))
def getStockDataVec(key):
    vec = []
    lines = open("BTC22.csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append(float(line.split(",")[0]))
        
        #print(vec)
    return vec
def sigmoid(x):
    return 1/(1+np.exp(-x))

def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        array1 = np.array(block[i])
        array2 = np.array(block[i + 1])
        res.append(sigmoid(np.subtract(array2, array1)))
    return np.array([res])


In [283]:
data = getStockDataVec(stock_name)
data

[[970.97,
  971.25,
  966.6,
  967.0,
  3.11991497,
  -0.38022155192561513,
  -0.3543368030674155,
  0.1617326540031245,
  1.6976011007739158,
  1.940942363560009,
  0.022483440311545408,
  1.5497461721368004,
  1.584383023916979,
  0.7173964263346092,
  0.10828650534574051,
  1.0513168834576088,
  0.0821450821664869,
  2.6944734285038927,
  1.0040330920372285,
  0.9959831084560717,
  0.0024811014968347533,
  404.6723172422072,
  401.42779718068357,
  -3.2445200615236445,
  -0.032445200615236446,
  0.9998970133882595,
  1.0040949702183986,
  0.0025166459038643814,
  397.3133494278592,
  398.98142550629876,
  1.668076078439583,
  -0.032445200615236446,
  1.0040330920372285,
  1.0010362329567628,
  0.0024584126551740705,
  408.4070629575151,
  407.1880409702347,
  -1.2190219872804278,
  -0.012190219872804277,
  1.0044157643822584,
  1.0005483931047328,
  0.002490615906352305,
  403.28007294119516,
  401.7272958679973,
  -1.5527770731978308,
  -0.015527770731978308,
  0.9996396841574271,


In [284]:
btc = [item[3] for item in data2]
btc

[967.0,
 968.18,
 969.0,
 971.05,
 966.6,
 966.6,
 970.97,
 970.57,
 966.6,
 965.08,
 963.99,
 963.93,
 963.75,
 968.999,
 962.54,
 971.75,
 965.02,
 968.18,
 967.25,
 968.979,
 962.87,
 971.87,
 971.55,
 968.938,
 971.47,
 967.91,
 967.0,
 970.36,
 969.96,
 966.11,
 966.1,
 966.01,
 969.04,
 966.01,
 964.04,
 965.89,
 964.05,
 965.51,
 970.01,
 964.15,
 965.59,
 970.01,
 970.61,
 970.61,
 970.54,
 970.61,
 965.53,
 963.55,
 970.68,
 963.6,
 963.66,
 970.69,
 970.69,
 971.04,
 963.87,
 971.04,
 966.789,
 966.789,
 966.79,
 968.999,
 968.998,
 969.0,
 968.999,
 971.27,
 969.0,
 971.95,
 965.59,
 964.52,
 964.8,
 964.88,
 964.7,
 971.95,
 965.59,
 966.21,
 971.34,
 971.05,
 966.5,
 970.94,
 966.089,
 971.17,
 966.299,
 971.0,
 970.87,
 971.38,
 964.73,
 970.72,
 966.35,
 965.123,
 964.81,
 965.84,
 964.8,
 971.11,
 965.84,
 970.08,
 965.979,
 965.52,
 969.68,
 970.1,
 969.03,
 969.72,
 965.5,
 965.11,
 969.19,
 969.14,
 963.6,
 969.04,
 965.1,
 965.1,
 965.1,
 965.888,
 967.05,
 964.96,


In [ ]:
stock_name = input("Enter Stock_name, Model_name")
model_name = input()
model = tf.keras.models.load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)


l = len(data) - 1
batch_size = 720
state = getState(data, 0, window_size + 1)
print(state)
total_profit = 0
agent.inventory = []
print(l)
for t in range(l):
    action = agent.act(state)
    price=btc[t]
    if t==0:
        Liquidity=200000
        
    elif t>=1:
        Liquidity=(200000-sum(agent.inventory)+total_profit)
    if 0>=Liquidity:
        action = 0 or 2
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # buy
        agent.inventory.append(btc[t])
        inventory=sum(agent.inventory)
        
        print("Buy: "+str(price)+" | Inventory: " + str(inventory)+" " + str(len(agent.inventory)))

    elif action == 2 and len(agent.inventory) > 0: # sell

        bought_price = window_size_price = agent.inventory.pop(0)
        inventory=sum(agent.inventory)
        total_profit += price - bought_price
        Liquidity=(200000-sum(agent.inventory)+total_profit)
        
        reward = max(((((inventory+Liquidity)/200000)-1)/(((price+.0001)/967)))-1, 0)

        print("Sell: " + str(price)+" Total Profit: "+str(total_profit)+" | Inventory: " + str(inventory)+" " + str(len(agent.inventory))+" | Liquidity: " + str(Liquidity)+" | Alpha: "+str((((inventory+Liquidity)/200000)/(price/46244.63))))
    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
        print("--------------------------------")
        print("Alpha: " + str((((Liquidity+inventory)/200000)/(data[t])/46244.63)))
        print("--------------------------------")
        print ("Total profit is:",formatPrice(total_profit))